首先我们导入一些必要的库，我们主要会用到Keras库。
- mnist: Keras库中包含的一个方便我们下载MNIST数据集的类
- Sequential: Keras线性模型框架，可以理解为积木的模板
- 神经网络中的一些常用层
    - Dense: 全连接层
    - Flatten: 平铺层
    - Conv2D: 二维卷积层
    - MaxPooling2D: 二维池化层

In [2]:
# 如果没有安装 keras 和 tensorflow 库
# 请使用 pip install keras tensorflow 安装
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


我们还是先导入MNIST数据集。但是为了能让卷积神经网络训练，我们还需要对数据做一些预处理和变换。回顾1.2中我们介绍了图像的矩阵表示与张量表示。在Keras中，图像数据需要以三阶张量的形式输入，而由于MNIST数据集是灰度图，图像是以矩阵形式表达的，所以我们需要将其进行一个形状变换。所以我们使用reshape函数将输入的每张图片从28x28的矩阵变形成28x28x1的张量。还有一点需要注意的是，原本的输入数据中每个像素是一个0-255的整数，但是对于神经网络的输入，我们一般希望将输入转化到0~1或者-1~1范围左右的数，所以我们将输入数据统一除以255。另外对于输出数据，我们不再简单的用一个数字来表示。正如在1.4中说到的，对于多分类问题，我们往往采用独热编码作为输出。Keras提供了一个函数 to_categorical 来完成这个变换。

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
width, height = x_train.shape[1], x_train.shape[2] # 获取图像的宽、高
n_train = x_train.shape[0] # 获取训练数据数量
n_test = x_test.shape[0] # 获取测试数据数量
x_train = x_train.reshape(n_train, width, height, 1) # 将输入转为三阶张量
x_test = x_test.reshape(n_test, width, height, 1) # 将输入转为三阶张量
print("reshape后的输入形状")
print(x_train.shape)
print(x_test.shape)
y_train = keras.utils.to_categorical(y_train) # 将输出转为独热编码
y_test = keras.utils.to_categorical(y_test) # 将输出转为独热编码
print("独热化后的输出形状")
print(y_train.shape)
print(y_test.shape)
print("处理前的最大值为%f" % x_train.max())
x_train = x_train / 255 # 将输入转化到0~1范围的数
x_test = x_test / 255 # 将输入转化到0~1范围的数
print("处理后的最大值为%f" % x_train.max())

11493376/11490434 [==============================] - 0s 0us/step
reshape后的输入形状
(60000, 28, 28, 1)
(10000, 28, 28, 1)
独热化后的输出形状
(60000, 10)
(10000, 10)
处理前的最大值为255.000000
处理后的最大值为1.000000


处理完数据后，我们开始搭建模型。我们使用Keras中的线性（Sequential）模型搭建一个基础的卷积神经网络，该网络的架构如下。
- 二维卷积层，32个5\*5的卷积核，使用ReLU作为激活函数
- 最大池化层，2\*2大小的池化核
- 二维卷积层，32个3\*3的卷积核，使用ReLU作为激活函数
- 最大池化层，2\*2大小的池化核
- 平铺层，将数据形状转为向量
- 全连接层，隐藏层维度为256，使用ReLU作为激活函数
- 全连接层，隐藏层维度为10，使用Softmax作为激活函数，输出每个分类的概率

在Keras的Sequential模块中，我们可以使用add函数，一层层地添加神经网络层。

In [4]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation="relu", input_shape=(width, height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(10, activation="softmax"))

Keras提供了summary函数，方便查看模型每一层的结构以及参数个数。

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               205056    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total para

搭建完卷积神经网络后，我们定义一个优化器，用来找到使损失函数最小的权重，这里我们使用Adam优化器。
最后我们使用交叉熵（一种用于计算多分类问题误差的函数，本书中不做深入解释，感兴趣的同学可以搜索相关信息）作为损失函数，使用准确率作为度量指标，并完成模型的搭建。

In [6]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

Keras提供了fit函数来进行训练，将训练的输入与输出x_train，y_train传给fit函数，指定批量大小为32，训练轮数为10轮。
一切就绪！开始训练（这会花上一些时间）。

In [7]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 27s 455us/step - loss: 0.1245 - acc: 0.9617 - val_loss: 0.0386 - val_acc: 0.9880
Epoch 2/5
60000/60000 [==============================] - 26s 441us/step - loss: 0.0407 - acc: 0.9872 - val_loss: 0.0347 - val_acc: 0.9899
Epoch 3/5
60000/60000 [==============================] - 26s 440us/step - loss: 0.0292 - acc: 0.9908 - val_loss: 0.0349 - val_acc: 0.9888
Epoch 4/5
60000/60000 [==============================] - 26s 439us/step - loss: 0.0215 - acc: 0.9932 - val_loss: 0.0300 - val_acc: 0.9908
Epoch 5/5
60000/60000 [==============================] - 26s 440us/step - loss: 0.0168 - acc: 0.9946 - val_loss: 0.0322 - val_acc: 0.9901


训练完毕！我们使用测试数据对训练好的模型进行测试。

In [8]:
score = model.evaluate(x_test, y_test)
print("损失为%f" % score[0])
print("准确度为%f" % score[1])

10000/10000 [==============================] - 1s 122us/step
损失为0.032209
准确度为0.990100


我们简单搭建的卷积神经网络在测试数据上取得了99.01%的准确度！是不是很棒！